## 1.파이썬과 MySQL활용

+ 파이썬과 mysql을 연결해주는 모듈 : mysqlclient, pymysql
    - pip install mysqlclient

In [1]:
! pip install mysqlclient

In [2]:
! pip install pymysql

In [86]:
# mysqlclient 모듈 사용
import MySQLdb

conn = MySQLdb.connect(host='127.0.0.1', user='root', password='1111', database='world')
print(conn)
conn.close()

<_mysql.connection open to '127.0.0.1' at 00000193AB71A840>


In [88]:
config = {'host':'127.0.0.1','user': 'root','password':'1111','database':'world'}

In [11]:
MySQLdb.connect(**config)

<_mysql.connection open to '127.0.0.1' at 00000193AAA77DD0>

#### (1) SELECT

In [22]:
conn = MySQLdb.connect(**config)

sql = 'select * from city limit 10'

cur = conn.cursor()
cur.execute(sql)

# print(cur)
# for data in cur:
#     #print(data)
#     print('%d %s %s %s %d'%data)
result = list()
for data in cur:
    result.append(data)
    
conn.close()

print(result)

[(1, 'Kabul', 'AFG', 'Kabol', 1780000), (2, 'Qandahar', 'AFG', 'Qandahar', 237500), (3, 'Herat', 'AFG', 'Herat', 186800), (4, 'Mazar-e-Sharif', 'AFG', 'Balkh', 127800), (5, 'Amsterdam', 'NLD', 'Noord-Holland', 731200), (6, 'Rotterdam', 'NLD', 'Zuid-Holland', 593321), (7, 'Haag', 'NLD', 'Zuid-Holland', 440900), (8, 'Utrecht', 'NLD', 'Utrecht', 234323), (9, 'Eindhoven', 'NLD', 'Noord-Brabant', 201843), (10, 'Tilburg', 'NLD', 'Noord-Brabant', 193238)]


#### (2) INSERT

In [44]:
conn = MySQLdb.connect(**config)

sql = 'create table if not exists tabltest(empno int,ename varchar(10),sal int, job varchar(10))'
cur = conn.cursor()
cur.execute(sql)

sql = 'insert into tabltest(empno,ename,sal) values(1,'scott',5000)'
cur.execute(sql)
conn.commit()

conn.close()

SyntaxError: invalid syntax (<ipython-input-44-6416cb8b5e4e>, line 7)

In [55]:
empno = 2
ename = 'miller'
sal = 1300

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = "insert into tabltest(empno, ename, sal) values('+str(empno)+','"+ename+"','+str(sal)+')"
cur.execute(sql)
conn.commit()

conn.close()

OperationalError: (1366, "Incorrect integer value: '+str(empno)+' for column 'empno' at row 1")

In [47]:
empno = 3
ename = 'tom'
sal = 2000

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = 'insert into tabltest(empno, ename, sal) values(%s, %s, %s)'
sql_data = (str(empno), ename,str(sal))

cur.execute(sql,sql_data)
conn.commit()

conn.close()

In [51]:
try: 
    conn =MySQLdb.connect(**config)
    cur = conn.cursor()
    
    empno = 3
    ename = 'tom'
    sal = 2000
    
    sql = 'insert into tabltest(empno, ename, sal) values(%s, %s, %s)'
    sql_data = (str(empno), ename,str(sal))

    
    cur.execute(sql,sql_data)
    conn.commit()
except Exception as e:
    print('erro :',e)
finally:
    conn.close()
        

In [54]:
# 데이터 여러 개 입력
conn = MySQLdb.connect(**config)
cur = conn.cursor()

for i in range(10):
    sql = 'insert into tabltest(empno,ename,sal) values(3,'jane',1000)'
    cur.execute(sql)
    conn.commit()
    
conn.close()

SyntaxError: invalid syntax (<ipython-input-54-af4444772650>, line 6)

#### (3) cursor 객체의 fetchall(), fetchmany(),fetchone()
    - default는 fetchall()

In [67]:
try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute('select * from tabltest limit 4')
    
    for i in cur:
        print(i)
    
finally:
    conn.close()

(3, 'tom', 2000, None)
(3, 'tom', 2000, None)
(3, 'jane', 1000, None)
(3, 'jane', 1000, None)


#### (4) DictCursor

+ 필드명을 키값으로 사용 가능

In [72]:
try: 
    conn = MySQLdb.connect(**config)
    cur = conn.cursor(MySQLdb.cursors.DictCursor)
    
    cur.execute('select * from tabltest')
    
    for i in cur.fetchall():
        print(i["ename"],i["sal"],i['ename'])

finally:
    conn.close()

tom 2000 tom
tom 2000 tom
jane 1000 jane
jane 1000 jane
jane 1000 jane
jane 1000 jane
jane 1000 jane
jane 1000 jane
jane 1000 jane
jane 1000 jane
jane 1000 jane
jane 1000 jane


#### (5) 응용 실습1: csv파일을 db에 저장

In [73]:
'''
CREATE TABLE IF NOT EXISTS suppliers(
    Supplier_Name    varchar(20),
    Invoice_Number   varchar(20),
    Part_Number      varchar(20),
    Cost             float,
    Purchase_Date    date
);
'''


'\nCREATE TABLE IF NOT EXISTS suppliers(\n    Supplier_Name    varchar(20),\n    Invoice_Number   varchar(20),\n    Part_Number      varchar(20),\n    Cost             float,\n    Purchase_Date    date\n);\n'

In [87]:
import csv

file = csv.reader(open("input.csv"))
print(file)

header = next(file)

try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute('delete from suppliers')
    conn.commit()
    
    for row in file:
        #print(row)
        cur.execute('insert into suppliers values(%s,%s,%s,%s,%s)',row)
        
    conn.commit()
finally:
    conn.close()

In [91]:
import csv
import datetime

file = csv.reader(open("input.csv"))
print(file)

header = next(file)

try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute('delete from suppliers')
    conn.commit()
    
    for row in file:
        #print(row)
        #cur.execute('insert into suppliers values(%s,%s,%s,%s,%s)',row)
        data = []
        for col in range(len(header)):
            if col == 4:
                tmp = datetime.datetime.strptime(row[col],'%Y-%m-%d')
                data.append(tmp)
            else:
                data.append(row[col])
                
        cur.execute('insert into suppliers values(%s,%s,%s,%s,%s)',data)
        
    conn.commit()
finally:
    conn.close()

#### (5) DB에서 가져온 데이터를 파일로 저장

In [97]:
try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute('select * from suppliers where cost > 500000')
    
#     for i in cur:
#         print(i)
    file = csv.writer(open('output.csv','w'),delimiter=',')
    
    header = ['Supplier Name','Invoice Number','Part Number','Cost','Purchase Date'] 
    file.writerow(header)
    
    for row in cur:
        file.writerow(row)
    
finally:
    conn.close()

## 2. 파이썬과 SQLite

1. 로컬DB로 사용할 수 있는 DBMS
    + Access
    + SQLite
    + Microsoft SQL Server Compact Edition
    + Firebird Embedded Server
    + H2
    + HSQL
    + Apache Derby
    + ...
    
2. SQLite
    + 파일 기반의 dbms, 저메모리, 빠른 처리속도
    + 오픈 소스
    + 별도의 db서버 없어도 쉽고 편리하게 사용할 수 있는 embedded sql db엔진
    + 안드로이드,아이폰, 등의 스마트 폰에 기본 내장된 db
    + 파이썬에도 내장
    + 표준sql 지원
    ---------------------------------------------
    + 컬럼 삭제하거나 변경하는 것등이 제한
    + left outer join 만 지원
    + 읽기 전용 뷰만 제공
    + ...
    
3. 공식 사이트
    + http://www.sqlite.org
    
4. 클라이언트 툴
    + http://www.sqliteexpert.com

In [1]:
import sqlite3

In [100]:
print(sqlite3.version)
print(sqlite3.sqlite_version)

2.6.0
3.33.0


In [101]:
sqlite3.connect('data/sqlitetest.db')

In [7]:
# select

conn = sqlite3.connect('data/sqlitetest.db')
cur = conn.cursor()

cur.execute('select * from tblbook')
for i in cur:
    print(i)

conn.close()

('최주현', '하늘과 땅')
('최주현', '바다')
('유은정', '바다')
('박성우', '문')
('최주현', '문')
('박성우', '천국')
('최지은', '천국')
('최주현', '천국')
('박성우', '고슴도치')
('서금동', '나')


In [6]:
conn = sqlite3.connect('data/sqlitetest.db')
cur = conn.cursor()

sql = ('select * from tblmember')
cur.execute(sql)

for i in cur:
    print(i)

(1, '홍길동', '2020-12-25')
(2, '임꺽정', '2021-04-27')


In [3]:
# 자동으로 db 생성하고 테이블 생성

conn = sqlite3.connect("data/auto.db")
# print(conn)
cur = conn.cursor()

cur.execute("create table if not exists tblmember(id   int, name varchar(10))")
conn.commit()

cur.execute("insert into tblmember values(1, '홍길동')")
conn.commit()

cur.execute("select * from tblMember")
for row in cur:
    print(row)

conn.close()

(1, '홍길동')
(1, '홍길동')


In [14]:
# csv파일을 읽어들여서 db에 저장
import csv

conn = sqlite3.connect('data/sqlitetest.db')
cur = conn.cursor()

# 테이블 조회
tables = cur.execute("select * from sqlite_master where type='table'")
for i in tables:
    print(i)
    
# 파일 불러오기
file = csv.reader(open('input.csv','r'),delimiter=',')
# for i in file:
#     print(i)

next(file)

for row in file:
    cur.execute('insert into suppliers values(?,?,?,?,?)',row)
    
conn.commit()


conn.close()

('table', 'tblBook', 'tblBook', 2, 'CREATE TABLE tblBook(\r\n                          author  varchar(20),\r\n                          title varchar(20)\r\n\t)')
('table', 'tblMember', 'tblMember', 3, 'CREATE TABLE tblMember(\r\n    id   int,\r\n    name  varchar(10),\r\n    regdate  datetime\r\n)')
('table', 'suppliers', 'suppliers', 4, 'CREATE TABLE suppliers(\r\n    Supplier_Name    varchar(20),\r\n    Invoice_Number   varchar(20),\r\n    Part_Number      varchar(20),\r\n    Cost             float,\r\n    Purchase_Date    date\r\n)')


In [21]:
################ Memory DB ##################
conn = sqlite3.connect(':memory:')
print(conn)
cur = conn.cursor()

cur.execute('create table phonebook(name varchar(10), phonenumber text);')
cur.execute('insert into phonebook values(?,?);',('tom','010-111-1111'))

# 여러 데이터 한번에 입력
datalist = (('jane','010-333-3333'),('jerry','010-444-4444'),('marry','010-555-5555'))
cur.executemany('insert into phonebook values(?,?)',datalist)


cur.execute('select * from phonebook')
for row in cur:
    print(row)
    
# 작업내용 스크립트 저장
f = open('dump.sql','w')
for line in conn.iterdump():
    f.write('{}\n'.format(line))
    
f.close()
    
conn.close()

('tom', '010-111-1111')
('jane', '010-333-3333')
('jerry', '010-444-4444')
('marry', '010-555-5555')


In [24]:
conn = sqlite3.connect('sqlitetest.db')
cur = conn.cursor()

file = open('dump.sql','r')
script = file.read()
print(script)

cur.executescript(script)

conn.close()

BEGIN TRANSACTION;
CREATE TABLE phonebook(name varchar(10), phonenumber text);
INSERT INTO "phonebook" VALUES('tom','010-111-1111');
INSERT INTO "phonebook" VALUES('jane','010-333-3333');
INSERT INTO "phonebook" VALUES('jerry','010-444-4444');
INSERT INTO "phonebook" VALUES('marry','010-555-5555');
COMMIT;



## 3. 파이썬과 MongoDB연동

    + pip install pymongo

In [1]:
import pymongo

### (1) 서버 연결

In [2]:
conn = pymongo.MongoClient('127.0.0.1',27017)
print(conn)
conn.close()

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)


### (2) DB연동

In [7]:
conn = pymongo.MongoClient('127.0.0.1',27017)

#dbname = conn.get_database('test')
#dbname = conn.test
dbname = conn['tset']
print(dbname)

conn.close()

Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'tset')


### (3) collection 연결

In [8]:
conn = pymongo.MongoClient('127.0.0.1',27017)

dbname = conn.test

collect = dbname.articles
print(collect)

conn.close()

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 'articles')


#### (4) 데이터 입력(1개)

In [14]:
conn = pymongo.MongoClient('127.0.0.1',27017)
dbname = conn.test
collect = dbname.people

# data = {'autor':'MIke','text':'my first','tag':['mongodb','python']}
# collect.insert_one(data)

# 입력하면서 id가져오기
# data = {'autor':'Tom','text':'my first','tag':['mongodb','python']}
# post = collect.insert_one(data).inserted_id
# print(post)

print(collect.count_documents({}))

for i in collect.find():
    print(i)

conn.close()

7
{'_id': ObjectId('608a5933e639169ac8b823cb'), 'name': 'Abet', 'age': 19.0, 'score': 10.0}
{'_id': ObjectId('608a5933e639169ac8b823cc'), 'name': 'Betty', 'age': 20.0, 'score': 10.0}
{'_id': ObjectId('608a5933e639169ac8b823cd'), 'name': 'Charlie', 'age': 23.0, 'skills': ['mongodb', 'nodejs', ['c', 'c++']]}
{'_id': ObjectId('608a5933e639169ac8b823ce'), 'name': 'David', 'age': 23.0, 'score': 20.0}
{'_id': ObjectId('608a6229e964309c86983daf'), 'name': 'Ellt', 'age': 17.0, 'score': 10.0}
{'_id': ObjectId('608b574c22c0a878a4545b85'), 'autor': 'MIke', 'text': 'my first', 'tag': ['mongodb', 'python']}
{'_id': ObjectId('608b588822c0a878a4545b87'), 'autor': 'Tom', 'text': 'my first', 'tag': ['mongodb', 'python']}


#### (5) 데이터 입력하기(여러 개)

In [16]:
conn = pymongo.MongoClient('127.0.0.1',27017)
dbname = conn.test
collect = dbname.people

print(collect.count_documents({}))

collect.insert_many([{'number' :i} for i in range(10)])

print(collect.count_documents({}))

conn.close()

7
17


In [17]:
conn = pymongo.MongoClient('127.0.0.1',27017)
dbname = conn.test
collect = dbname.people

data = list()
data.append({'name':'aron','age':20})
data.append({'name':'tom','age':21})
data.append({'name':'jane','age':22})
data.append({'name':'jerry','age':23})

collect.insert_many(data)

conn.close()

#### (6) 조회

In [22]:
conn = pymongo.MongoClient('127.0.0.1',27017)
dbname = conn.test
collect = dbname.people

# # 전체 조회
# for i in collect.find():
#     print(i)
    
# # 나이가 20살인 도큐먼트 조회
# for i in collect.find({'age':20}):
#     print(i)
    
# 나이가 23세 이상인 도큐먼트
for i in collect.find({'age':{'$gte':23}}):
    print(i)

print('-----------------------------------')

print(collect.find_one())
    
conn.close()

{'_id': ObjectId('608a5933e639169ac8b823cd'), 'name': 'Charlie', 'age': 23.0, 'skills': ['mongodb', 'nodejs', ['c', 'c++']]}
{'_id': ObjectId('608a5933e639169ac8b823ce'), 'name': 'David', 'age': 23.0, 'score': 20.0}
{'_id': ObjectId('608b5e8922c0a878a4545b9a'), 'name': 'jerry', 'age': 23}
-----------------------------------
{'_id': ObjectId('608a5933e639169ac8b823cb'), 'name': 'Abet', 'age': 19.0, 'score': 10.0}


#### (7) 수정

In [29]:
conn = pymongo.MongoClient('127.0.0.1',27017)
dbname = conn.test
collect = dbname.people

print(collect.find_one({'athor':'tom'}))

#collect.update_one()

conn.close()

None
